In [1]:
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime
import threading
from typing import Dict, Optional
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.common import BarData
from ib_insync import *

import vectorbt as vbt

# Load environment variables from .env file
load_dotenv() #Comment out if default localhost IP is sufficient
# Get IP from .env or use default
IB_IP = os.getenv("IB_IP", "127.0.0.1")

In [2]:
class TradingApp(EClient, EWrapper):

    def __init__(self) -> None:
        EClient.__init__(self, self)
        self.data: Dict[int, pd.Dataframe] = {}

    def error(self, reqId: int, errorCode: int, errorString: str, advanced: dict ={}):# -> None:
        print(f"Error: {reqId}, {errorCode}, {errorString}")

    def nextValidId(self, orderId: int) -> None:
        super().nextValidId(orderId)
        self.nextOrderId = orderId

    def get_historical_data(self, reqId: int, contract: Contract) -> pd.DataFrame:
        self.data[reqId] = pd.DataFrame(columns=["time", "high", "low", "close"])
        self.data[reqId].set_index("time", inplace=True)
        self.reqHistoricalData(
            reqId=reqId,
            contract=contract,
            endDateTime="",
            durationStr="1 D",
            barSizeSetting="1 min",
            whatToShow="MIDPOINT",
            useRTH=0,
            formatDate=2,
            keepUpToDate=False,
            chartOptions=[],
        )
        time.sleep(5)
        return self.data[reqId]

    def historicalData(self, reqId: int, bar: BarData) -> None:
        df = self.data[reqId]
        df.loc[
            pd.to_datetime(bar.date, unit="s"),
            ["high", "low", "close"]
        ] = [bar.high, bar.low, bar.close]
        df = df.astype(float)
        self.data[reqId] = df
        
    @staticmethod                                                                                         
    def get_contract(symbol: str) -> Contract:
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "STK"
        contract.exchange = "SMART"
        contract.currency = "USD"
        return contract

    def place_order(self, contract: Contract, action: str, order_type: str, quantity: int) -> None:
        order = Order()
        order.action = action
        order.orderType = order_type
        order.totalQuantity = quantity
        self.placeOrder(self.nextOrderId, contract, order)
        self.nextOrderId += 1
        print("Order placed")        

In [3]:
app = TradingApp()


In [4]:
app.connect(IB_IP,7497, clientId=1)


In [5]:
threading.Thread(target=app.run, daemon=True).start()


Error: -1, 2104, Market data farm connection is OK:usfarm.nj
Error: -1, 2104, Market data farm connection is OK:hfarm
Error: -1, 2104, Market data farm connection is OK:jfarm
Error: -1, 2104, Market data farm connection is OK:usfuture
Error: -1, 2104, Market data farm connection is OK:cashfarm
Error: -1, 2104, Market data farm connection is OK:eufarmnj
Error: -1, 2104, Market data farm connection is OK:usfarm
Error: -1, 2106, HMDS data farm connection is OK:euhmds
Error: -1, 2106, HMDS data farm connection is OK:fundfarm
Error: -1, 2106, HMDS data farm connection is OK:ushmds
Error: -1, 2158, Sec-def data farm connection is OK:secdefnj


In [6]:

nvda=TradingApp.get_contract("NVDA")
nvda


Contract(secType='STK', symbol='NVDA', exchange='SMART', currency='USD')

In [7]:
data=app.get_historical_data(99,nvda)
data

,high,low,close
time,,,
2025-05-14 08:00:00,132.56,131.97,132.38
2025-05-14 08:01:00,132.38,132.14,132.16
2025-05-14 08:02:00,132.25,132.02,132.02
2025-05-14 08:03:00,132.03,131.63,131.63
2025-05-14 08:04:00,132.00,131.63,131.97
...,...,...,...
2025-05-14 13:48:00,133.09,132.62,132.65
2025-05-14 13:49:00,133.18,132.41,133.16
2025-05-14 13:50:00,133.22,132.77,132.81


In [8]:
type(data)

pandas.core.frame.DataFrame